# SudokuPlantDesign.jl - Basic Example

This notebook serves as a basic usage guide to the module `SudokuPlantDesign.jl`. It is devided into the following general parts:
1. Importing modules
2. Generating an initial check distribution
3. Defining the optimization parameters
4. Performing the optimization

Each part comes with code and examples.

### 1) Installing and Importing modules

If not done already, install the modules `SudokuPlantDesign` into the package installation

In [ ]:
]add BlockArrays ProgressMeter PyPlot Statistics "https://github.com/janattig/SudokuPlantDesign.jl"

To utilize `SudokuPlantDesign` functions within the notebook (and also `PyPlot` functions, which come in handy later), use

In [ ]:
using SudokuPlantDesign
using PyPlot

### 2) Generate initial (random) check distribution

To start a new plant design, the first step is to generate a new configuration of checks, distributed over the available are. Start by generating such a new configuration `conf`. In this example, it consists of 2 horizontal and 5 vertical blocks of dimensions `10` x `2` respectively. In total, there are `4` different types of checks in the configuration.

As an optional argument, the boundary conditions (present in the optimization algorithm later on) can be passed as an argument `bc`. Values are either `:periodic` or `open`. 

In [ ]:
conf = get_configuration(
    [10,10], [2,2,2,2,2],  4
    ;
    bc = :open
);

To visualize the configuration `conf`, use `show_configuration`. If you are seeing a uniform color pattern, remember that initially all plots are assumed to be entries.

In [ ]:
show_configuration(conf)

You can designate empty plots which will be ignored in the sudoku optimization (missing plots, etc). In this example, leave the plot `[1,3]` and `[1,4]` empty. Configuration is plotted again to show the update.

In [ ]:
empty_plots!(conf, 1:1,3:4)
show_configuration(conf)

For moving on to the actual optimization, the different checks are initialized randomly with one check per check type per block.

In [ ]:
initialize_checks_per_block!(conf)
show_configuration(conf)

At this point, saving the resulting configuration might be a good idea. This can be done by utilizing the `PyPlot` command `savefig` which saves the current figure as an image file.

In [ ]:
show_configuration(conf)

mkpath("output/")
savefig("output/initial_random_check_distribution.png")

### 3) Define optimization parameters

In order to optimize the check distribution, one has to specify two aspects explicitly:

- cost functions / what are good or bad check distributions?
- updates / how do you generate a new check distribution from an old one?

#### Cost functions

The *cost function* defines a function that associates a value to every distribution of checks, i.e. it measures how well the checks are distributed. In `SudokuPlantDesign`, it is implemented as a julia function that takes the configuration `conf` as an input and returns a numeric value. Lower values represent lower costs and are therefore desirable. If you want to associated an interpretation to the actual cost value, you can think of it as *how far is the check distribution still away from being perfect*.

In practice, the overall cost function can be built from small, already implemented pieces in e.g. the following way:

In [ ]:
function K_indiv(conf :: C) :: Float64 where {C <: CheckConfiguration}
    return  K_checks_per_type_per_block(conf, 1)*20+
            K_neighbors_different_check_functional(conf, d->1/(d^3))*0.5 +
            K_neighbors_same_check_functional(conf, d->1/(d^3))
end

Here, the following partial cost functions are used:
- `K_checks_per_type_per_block` : Gives costs if there are checks which occur more than `1` time per check type and block. Optimizes to get an exact amount of checks per type per each block. Additionally, here it is weighted by a factor of `20`.
- `K_neighbors_different_check_functional` : Gives costs when checks of different types are neighbors, i.e. it acts as a repulsive potential of different neighboring checks (e.g. check 1 and check 3) with the functional dependence $\text{costs} \sim 1/\text{distance}^3$. Additionally, here it is weighted by a factor of `0.5`.
- `K_neighbors_same_check_functional` : Gives costs when checks of the same check type are neighbors, i.e. it acts as a repulsive potential of neighboring checks of equal type (e.g. check 1 and check 1) with the functional dependence $\text{costs} \sim 1/\text{distance}^3$. 

Summarizing, one could say that the cost function defines the *goal* of the optimization in terms of judging what is *good* and *bad*.

### Define update moves.
Select which update moves you want to occur. `UpdateNewCheckLabel` puts new checks into the design, `UpdateSwapCheckCheck` swaps checks with other checks, and `UpdateSwapCheckEntry` swaps checks with entries. Run `100000` updates.

In [ ]:
updates = [
    UpdateNewCheckLabel(),
    UpdateSwapCheckCheck(),
    UpdateSwapCheckEntry()
]

## 4) Run sudoku optimization

In [ ]:
kosten = optimize_design!(
    C,
    updates,
    K_indiv,
    100000,
    beta
)

In [ ]:
kosten .-= minimum(kosten)
kosten .+= 1

#print things
println("Kosten: ",K_indiv(C)+kosten[1], " -> ", K_indiv(C)+kosten[end])
println("perc_checks: ",C.num_checks_total / (C.num_plots_total - C.num_checks_total))
println("entries: ",C.num_plots_total - C.num_checks_total)
println("checks:    ",C.num_checks_total, "  (", round(100*C.num_checks_total / C.num_plots_total, digits=2), "%)")
for i in 1:C.N
    println("check ($(i)): ",C.num_checks[i], "  (", round(100*C.num_checks[i] / C.num_checks_total, digits=2), "%)")    
end

show_configuration(C,cmap=cmap_sudoku,zoom=0.2)


figure()
plot(kosten)
yscale("log")


#save without check labels
show_configuration(C,check_labels=false,cmap=cmap_sudoku)
#savefig("final_sudoku_design_"*trialname*".png")
savefig("final_sudoku_design_"*trialname*".pdf")

## 6) Plot with Plant_Codes and write data to xlsx

## 5) Save field plan and design data

In [ ]:
templatename="Sample_data_Sudoku-augmented.xlsx"

#read in check data
checkdata = DataFrame(XLSX.readtable(templatename, "checks")...)
checknames=names(checkdata)
for i in 1:length(checknames)
    checkdata[!,Symbol(checknames[i])]=string.(collect(checkdata[!,Symbol(checknames[i])]))
end
if nrow(checkdata)!= numberofchecktypes
    println("Number of checks in csv not matching pre-defined checks!")
end


#replace missing with NA
for col in eachcol(checkdata)
    replace!(col,missing => "NA")
end


#depending on reading in additionally entry file
if  designtype=="checks"
    print("Only checks were successfully read in, as specified")
    
elseif designtype=="all"
    #read in entry data
    entrydata = DataFrame(XLSX.readtable(templatename, "entries")...)
    genonames=names(entrydata)
    for i in 1:length(checknames)
        entrydata[!,Symbol(genonames[i])]=string.(collect(entrydata[!,Symbol(genonames[i])]))
    end
    #replace missing with NA
    for col in eachcol(entrydata)
        replace!(col,missing => "NA")
    end
    
    #catch if not matching number of entries
    if nrow(entrydata)!= numberofentries
        println("Number of entries in xlsx datafile not matching pre-defined number of entries!")
    end
    
    
    #shuffle entrydata?
    if entries_randomized=="yes"
        entrydata = entrydata[shuffle(1:nrow(entrydata)), :]
        entrydata[!,:entry]=string.(collect(entrydata[!,:entry]))
        print("Entries are randomized ")
        #print(entrydata)

    elseif entries_randomized=="no"
        print("Entries are not randomized")
        #print(entrydata)

    else
        print("Specify if entries are randomized or sorted with entries_randomized yes/no!")

    print("entries and checks are successfully read in")
    end
    
else
    print("Please specify designtype (checks/all)!")
        
end

#get the Plant label
#POS = get_POS_block_x(C,posempty=true) #if over 2 big blocks in splitted by x
POS = get_POS_coordinatewise(C,"x","left","lower","snake") #with robot

if designtype=="checks"
    L = get_Plant_Code_Label_allinfo_checksonly(C,POS,checkdata)
else
    L = get_Plant_Code_Label_allinfo(C,POS,entrydata,checkdata)
end   

show_pos_and_labels_allinfo(C, POS, L, zoom=0.9,fontsize=7,alpha=0.5,cmap=cmap_sudoku)
savefig("final_sudoku_design_labels_"*trialname*".pdf")

In [ ]:
#write file to xlsx
write_to_xlsx_allinfo(C.configuration,trialname,POS,L)


#load_configuration(C, "configuration_"*trialname*".txt")
save_configuration(C, "configuration_"*trialname*".txt")
